In [1]:
import (
    "encoding/csv"
    "fmt"
    "os"
    "math"
    "sort"
    "strconv"
    "strings"
)

import (
    "gonum.org/v1/plot"
    "gonum.org/v1/plot/plotter"
    "gonum.org/v1/plot/plotutil"
    "gonum.org/v1/plot/vg"
    "gonum.org/v1/gonum/stat"
)

In [ ]:
func ReadCSV(path string) ([][]string, error) {
    csvFile, err := os.Open(path)
    if err != nil {
        return [][]string{}, err
    }
    defer csvFile.Close()

    lines, err := csv.NewReader(csvFile).ReadAll()
    if err != nil {
        return [][]string{}, err
    }

    return lines, nil
}

func ParseInt(s string) int {
    i, err := strconv.Atoi(s)
    if len(s) > 0 && err != nil {
        fmt.Printf("Failed to convert string %s to an int\n", s)
    }
    
    return i
}

func ParseFloat(s string) float64 {
    i, err := strconv.ParseFloat(s, 64)
    if len(s) > 0 &&  err != nil {
        fmt.Printf("Failed to convert string %s to a float\n", s)
    }
    
    return i
}

In [7]:
type Station struct {
    Ward int
    Name string
    Voters int
    Votes int
    Turnout float64
}

csv, err := ReadCSV("./data/voter_stats_2018.csv")
var data = make([]Station, 0)

for i, line := range csv[1:] {
    if strings.HasSuffix(line[0], "Total") {
        continue
    }
    
    data = append(data, Station{
        Ward: ParseInt(line[0]),
        Name: line[2],
        Voters: ParseInt(line[13]),
        Votes: ParseInt(line[14]),
        Turnout: ParseFloat(line[14]) / ParseFloat(line[13]),
    })
}

In [8]:
var stations = make(map[int] int)
var voters = make(map[int] int)
var votes = make(map[int] int)

var turnouts = make(map[int] float64) 

for _, r := range data {
    stations[r.Ward] += 1
    
    voters[r.Ward] += r.Voters
    votes[r.Ward] += r.Votes
}

for k := range voters {
    turnouts[k] = float64(votes[k]) / float64(voters[k])
}

In [10]:
pts := make(plotter.XYs, len(stations))

for k, v := range turnouts {
    idx := k - 1
    
    pts[idx].X = float64(k)
    pts[idx].Y = v
}

p, err := plot.New()
if err != nil {
    panic(err)
}

// p.Title.Text = title
p.X.Label.Text = "Ward #"
p.Y.Label.Text = "Turnout"

// p.HideX()

err = plotutil.AddLinePoints(p, pts)
if err != nil {
    panic(err)
}

p.Y.Min = 0
p.Y.Max = p.Y.Max * 1.25

// if err := p.Save(6*vg.Inch, 6*vg.Inch, "./visualizations/1.png"); err != nil {
//     panic(err)
// }

In [12]:
pts := make(plotter.XYs, len(stations))

for i, _ := range stations {
    idx := i - 1
    
    pts[idx].X = float64(stations[i])
    pts[idx].Y = turnouts[i]
}

p, err := plot.New()
if err != nil {
    panic(err)
}

// p.Title.Text = title
p.X.Label.Text = "# of Stations"
p.Y.Label.Text = "Turnout"

s, err := plotter.NewScatter(pts)
if err != nil {
    panic(err)
}

p.Add(plotter.NewGrid())
p.Add(s)

p.X.Min = 0
p.X.Max = p.X.Max * 1.25
p.Y.Min = 0
p.Y.Max = p.Y.Max * 1.25

// if err := p.Save(6*vg.Inch, 6*vg.Inch, "./visualizations/2.png"); err != nil {
//     panic(err)
// }

In [13]:
p, err := plot.New()
if err != nil {
    panic(err)
}

var max = make(map[int] float64)
var min = make(map[int] float64)

w := vg.Points(5)
for ward, stations := range stations {
    pts := make(plotter.Values, stations)
    values := make([]float64, 0)

    i := 0
    for _, row := range data {
        if row.Ward == ward && !math.IsInf(row.Turnout, 0) {
            pts[i] = row.Turnout
            values = append(values, row.Turnout)
            
            i += 1
        }
    }
    
    b, err := plotter.NewBoxPlot(w, float64(ward - 1), pts)
    if err != nil {
        panic(err)
    }
    
    p.Add(b)
    
    sort.Slice(values, func(i, j int) bool {
        return values[i] < values[j]
    })

    max[ward] = stat.Quantile(0.91, stat.Empirical, values, nil)
    min[ward] = stat.Quantile(0.09, stat.Empirical, values, nil)
}

p.X.Label.Text = "Turnout per Voting Station"

// if err := p.Save(6*vg.Inch, 6*vg.Inch, "./visualizations/3.png"); err != nil {
//     panic(err)
// }

In [16]:
func ContainKeywords(s string) bool {
    keywords := [8]string{
        "retirement", "care", "nursing", "senior", "healthcare", "residence", "addiction", "mental health",
    }
    
    for _, v := range keywords {
        if strings.Contains(strings.ToLower(s), v) {
            return true
        }
    }
    
    return false
}

highOutliers := make([]Station, 0)
lowOutliers := make([]Station, 0)

for i, row := range data {
    if math.IsInf(row.Turnout, 0) {
        continue
    }
    
    if row.Turnout <= min[row.Ward] {
        lowOutliers = append(lowOutliers, row)
    } else if row.Turnout >= max[row.Ward] {
        highOutliers = append(highOutliers, row)
    }
}

for i, station := range highOutliers {
    if ContainKeywords(station.Name) {
//         fmt.Println(station.Name)
    }
}

In [17]:
type Ward struct {
    ID int
    Population int
    PostSecondary int
    MedianIncome int
}

csv, err := ReadCSV("./data/ward_profile_2016.csv")
var data = make([]Ward, 0)

for i, line := range csv[1:] {
    data = append(data, Ward{
        ID: ParseInt(line[0]),
        Population: ParseInt(line[1]),
        PostSecondary: ParseInt(line[2]),
        MedianIncome: ParseInt(line[3]),
    })
}

In [36]:
var postsecondary = make(plotter.XYs, len(data))
var medianIncome = make(plotter.XYs, len(data))

var truncatedTurnout = make(plotter.XYs, len(data))

for i, r := range data {
    postsecondary[i].X = float64(r.ID)
    medianIncome[i].X = float64(r.ID)
    truncatedTurnout[i].X = float64(r.ID)
    
    postsecondary[i].Y = float64(r.PostSecondary) / float64(r.Population)
    medianIncome[i].Y = float64(r.MedianIncome) / float64(r.Population)
    truncatedTurnout[i].Y = turnouts[r.ID]
}

p, err := plot.New()
if err != nil {
    panic(err)
}

// p.Title.Text = title
p.X.Label.Text = "Ward #"
// p.Y.Label.Text = "Turnout"

// p.HideX()

err = plotutil.AddLinePoints(
    p,
    "Turnouts", truncatedTurnout,
    "% Pop. with Post-Secondary Degrees", postsecondary,
    "% Pop. with >$90,000 Annual Household Income", medianIncome,
)
if err != nil {
    panic(err)
}

p.X.Min = 0
p.X.Max = 25
p.Y.Min = 0
p.Y.Max = p.Y.Max * 1.25

// if err := p.Save(6*vg.Inch, 6*vg.Inch, "./visualizations/4.png"); err != nil {
//     panic(err)
// }